In [2]:
import glob
from IPython.display import display
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns 
import re
from sklearn.preprocessing import LabelEncoder

C:\Users\ishizuka\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\ishizuka\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\ishizuka\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [60]:
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

path_in_trains = "../data/raw/train/*.csv"
path_in_test = "../data/raw/test.csv"

dfs = []
for fpath in sorted(glob.glob(path_in_trains)):
    _df = pd.read_csv(fpath)
    dfs.append(_df)
train_df = pd.concat(dfs)
test_df = pd.read_csv(path_in_test)
all_df = pd.concat([train_df, test_df]).reset_index(drop=True)

C:\Users\ishizuka\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [138]:
def change_to_num(all_df):
    """数値変数に変換
    """

    # 値の種類が1種類以下のものを除外　
    drop_cols = all_df.nunique()[all_df.nunique() <= 1].index
    all_df = all_df.drop(columns=drop_cols)

    # 最寄駅：距離（分）
    d = {
        "30分?60分": 45,
        "1H?1H30": 75,
        "2H?": 120,
        "1H30?2H": 105,
    }
    all_df["最寄駅：距離（分）"] = all_df["最寄駅：距離（分）"].replace(d)
    all_df["最寄駅：距離（分）"] = all_df["最寄駅：距離（分）"].astype(float)

    # 面積
    all_df.loc[:, '面積（㎡）'] = all_df["面積（㎡）"].replace({'2000㎡以上': "2000", "5000㎡以上": "5000"})
    all_df.loc[:, '面積（㎡）'] = all_df["面積（㎡）"].astype(int)
    all_df.loc[:, "面積_2000"] = all_df["面積（㎡）"] == 2000

    # 建築年
    d = {}
    for val in all_df["建築年"].value_counts().keys():
        if "平成" in val:
            _year = int(val.split("平成")[1].split("年")[0])
            year = 1988 + _year
        if "昭和" in val:
            _year = int(val.split("昭和")[1].split("年")[0])
            year = 1925 + _year
        if "令和" in val:
            _year = int(val.split("令和")[1].split("年")[0])
            year = 2018 + _year
        d[val] = year
    d["戦前"] = 1945
    all_df.loc[:, "建築年"] = all_df["建築年"].replace(d)

    # 取引時点
    all_df["取引年"] = all_df["取引時点"].apply(lambda x : int(x[:4]) if type(x)==str else np.nan)
    all_df["取引四半期"] = all_df["取引時点"].apply(lambda x : int(x[6]) if type(x)==str else np.nan)
    
    all_df.loc[:, "取引時点"] = all_df["取引時点"].apply(lambda x : re.sub('年第１四半期', '.00', x))
    all_df.loc[:, "取引時点"] = all_df["取引時点"].apply(lambda x : re.sub('年第２四半期', '.25', x))
    all_df.loc[:, "取引時点"] = all_df["取引時点"].apply(lambda x : re.sub('年第３四半期', '.50', x))
    all_df.loc[:, "取引時点"] = all_df["取引時点"].apply(lambda x : re.sub('年第４四半期', '.75', x))
    all_df.loc[:, "取引時点"] = all_df["取引時点"].apply(float)


    return all_df

In [114]:
def na_prep(all_df):
    """欠損値の処理
    """
    
    # 欠損値の数
    all_df.loc[:, "na_num"] = all_df.isnull().sum(axis=1).values
    
    na_cols = list(all_df.isnull().sum()[all_df.isnull().sum()>0].index)
    na_cols.remove("取引価格（総額）_log")
    # 欠損かどうかを表す二値変数
    for col in na_cols:
        all_df.loc[:, "{}_isna".format(col)] = all_df[col].isnull()
    
    # 欠損値の補完
    na_obj_col = all_df[na_cols].dtypes[all_df[na_cols].dtypes=="object"].index
    na_num_col = set(na_cols) - set(na_obj_col)
    for col in na_obj_col:
        all_df.loc[:,col] = all_df[col].fillna(all_df[col].mode()[0])
    for col in na_num_col:
        all_df.loc[:,col] = all_df[col].fillna(all_df[col].mean())
        
    return all_df

In [115]:
def madori_prep(all_df):
    """「間取り」の処理
    """
    all_df['L'] = all_df['間取り'].map(lambda x: 1 if 'Ｌ' in str(x) else 0)
    all_df['D'] = all_df['間取り'].map(lambda x: 1 if 'Ｄ' in str(x) else 0)
    all_df['K'] = all_df['間取り'].map(lambda x: 1 if 'Ｋ' in str(x) else 0)
    all_df['S'] = all_df['間取り'].map(lambda x: 1 if 'Ｓ' in str(x) else 0)
    all_df['R'] = all_df['間取り'].map(lambda x: 1 if 'Ｒ' in str(x) else 0)
    all_df['OpenFloor'] = all_df['間取り'].map(lambda x: 1 if 'オープンフロア' in str(x) else 0)
    all_df['RoomNum'] = all_df['間取り'].map(lambda x: re.sub("\\D", "", str(x)))
    all_df.loc[:,'RoomNum'] = all_df['RoomNum'].map(lambda x:int(x) if x!='' else 0)
    all_df['TotalRoomNum'] = all_df[['L', 'D', 'K', 'S', 'R', 'RoomNum']].sum(axis=1)
    all_df['RoomNumRatio'] = all_df['RoomNum'] / all_df['TotalRoomNum']   
    
    return all_df

In [139]:
df = change_to_num(all_df)
df = na_prep(df)
df = madori_prep(df)

In [5]:
def get_labelencoding(all_df):
    cols = all_df.dtypes[all_df.dtypes=="object"].index
    for col in cols:
        all_df.loc[:, col] = all_df[col].fillna("NaN")
        le = LabelEncoder()
        all_df.loc[:, col] = le.fit_transform(all_df[col])

    return all_df

In [ ]:
###
def preprocess(df, flag):
    
    # 用途
    col = "用途"
    d = {}
    for val in df[col].value_counts().keys():
        if "、" in val:
            d[val] = val.split("、")[0]
    df[col] = df[col].replace(d)

    return df